# Load and test trained model

## Load libraries

In [100]:
import numpy as np
import torch

import torch.nn as nn
from enduro_lstm import *

In [101]:
device = conf_cuda(False)

Selected CPU


In [102]:
device

device(type='cpu')

In [103]:
device.type

'cpu'

## Set configurations

In [ ]:
import os
dir_path = "best_models/chuncked_sequence/elman/tanh_softmax/elman_tanh_softmax_m35to35_f1to120_H200_epoch10000" + "/"
arr = os.listdir(f'./{dir_path}')
for i in range(len(arr)):
    print(arr[i])

In [ ]:
model_path = dir_path + "teste_com_rnn_m35to35_f1to120_epoch10000_H200"

In [ ]:
start_match = 45
end_match = 50

hidden_neurons = 200
zigzag = False
is_softmax = True

start_frame = 1
end_frame = 120

In [ ]:
data_path = r"../1-generate/data/"

use_cuda = False
load_checkpoint = True

## Load trained model

In [ ]:
if zigzag:
    output_size = 2
else:
    output_size = 9

In [105]:
path = 'best_models/raw_sequence/lstm/raw_sequences/170x120/1-2-zigzag_m35to35_f1to120_epoch4240_H200_sig/'
newpath = path + 'zigzag_m35to35_f1to120_epoch10000_H200'

In [106]:
hidden_neurons = 200
load_checkpoint = True

In [107]:
if zigzag:
    output_size = 2
else:
    output_size = 9

In [108]:
def load_checkpoint(model, filename='checkpoint.pth.tar'):
    
    print("=> loading checkpoint '{}'".format(filename))
    checkpoint = torch.load(filename, map_location=device)
    model.load_state_dict(checkpoint['state_dict'])

    return model, checkpoint['optimizer']['state'][0]['step'], checkpoint['losslogger']

In [28]:
if load_checkpoint:
    model = Model1(device=device, input_size=20400, output_size=output_size, hidden_dim=hidden_neurons, n_layers=1, is_softmax=is_softmax)
    model, last_epoch, last_logger = load_checkpoint(model, model_path)
else:
    model = Model(device=device, input_size=20400, output_size=output_size, hidden_dim=hidden_neurons, n_layers=1, is_softmax=is_softmax)
    model.load_state_dict(torch.load(model_path, map_location=device))
model.eval()

=> loading checkpoint 'best_models/raw_sequence/lstm/raw_sequences/170x120/1-2-zigzag_m35to35_f1to120_epoch4240_H200_sig/zigzag_m35to35_f1to120_epoch10000_H200'


Model(
  (lstm): LSTM(20400, 200, batch_first=True)
  (fc): Linear(in_features=200, out_features=2, bias=True)
  (out): Sigmoid()
)

In [11]:
model = Model(device=device, input_size=20400, output_size=output_size, hidden_dim=hidden_neurons, n_layers=1, is_softmax=is_softmax)
torch.load(newpath, map_location=device)
model.eval()

Model(
  (lstm): LSTM(20400, 500, batch_first=True)
  (fc): Linear(in_features=500, out_features=9, bias=True)
  (out): Softmax(dim=None)
)

In [60]:
ACTIONS_LIST = get_actions_list(zigzag=zigzag)

In [33]:
num_of_frames_arr = []
frames_arr = []
actions_arr = []

start_frame = 1
end_frame = 1000

for m in range(start_match, end_match + 1):
    
    num_of_frames, frames, actions, rewards, lifes = load_npz(data_path, m)
    frames = frames[start_frame - 1:end_frame]
    actions = actions[start_frame - 1:end_frame]
    
    action_one_hot = [prepare_action_data(i, ACTIONS_LIST) for i in actions]
    actions = np.array(action_one_hot)
    actions = actions.reshape(len(actions), -1)
    
    frames_arr.append(frames)
    actions_arr.append(actions)
    num_of_frames_arr.append(end_frame - start_frame + 1) 

X_train = np.array(frames_arr)/255
Y_train = np.array(actions_arr)
num_of_frames_arr = np.array(num_of_frames_arr)

X_train = torch.tensor(X_train).float()
Y_train = torch.tensor(Y_train).float()

Successfully loaded NPZ.
Successfully loaded NPZ.
Successfully loaded NPZ.
Successfully loaded NPZ.
Successfully loaded NPZ.
Successfully loaded NPZ.


In [77]:
#out, h = model(X_train)

In [78]:
"""criterion = nn.MSELoss()
loss = criterion(out, Y_train.view(-1,len(ACTIONS_LIST)).float())
loss"""

'criterion = nn.MSELoss()\nloss = criterion(out, Y_train.view(-1,len(ACTIONS_LIST)).float())\nloss'

In [79]:
"""out, h = model(X_train[0].reshape(1, 1000, 20400))
loss = criterion(out, Y_train[0].view(-1,len(ACTIONS_LIST)).float())
out, h = model(X_train[1].reshape(1, 1000, 20400))
loss = criterion(out, Y_train[1].view(-1,len(ACTIONS_LIST)).float())
loss.item()"""

'out, h = model(X_train[0].reshape(1, 1000, 20400))\nloss = criterion(out, Y_train[0].view(-1,len(ACTIONS_LIST)).float())\nout, h = model(X_train[1].reshape(1, 1000, 20400))\nloss = criterion(out, Y_train[1].view(-1,len(ACTIONS_LIST)).float())\nloss.item()'

## Prepare cell with trained model 

In [80]:
lstmcell = nn.LSTMCell(20400, hidden_neurons)
linear = nn.Linear(hidden_neurons, output_size)
if is_softmax:
    output = nn.Softmax()
else:
    output = nn.Sigmoid()

In [81]:
output

Sigmoid()

In [82]:
lstmcell.weight_ih = model.lstm.weight_ih_l0
lstmcell.weight_hh = model.lstm.weight_hh_l0
lstmcell.bias_hh = model.lstm.bias_hh_l0
lstmcell.bias_ih = model.lstm.bias_ih_l0
linear.weight = model.fc.weight
linear.bias = model.fc.bias

In [83]:
hx = torch.zeros(1, hidden_neurons)
cx = torch.zeros(1, hidden_neurons)

In [84]:
print(model.lstm.weight_ih_l0.shape)
print(model.lstm.weight_hh_l0.shape)
print(model.lstm.bias_ih_l0.shape)
print(model.lstm.bias_hh_l0.shape)
print(model.fc.weight.shape)
print(model.fc.bias.shape)

torch.Size([800, 20400])
torch.Size([800, 200])
torch.Size([800])
torch.Size([800])
torch.Size([2, 200])
torch.Size([2])


## Plot graphs

In [85]:
"""train_loss_arr = np.load(path + 'train_loss_arr.npz')
valid_loss_arr = np.load(path + 'valid_loss_arr.npz')
valid_loss_mean_arr = np.load(path + 'valid_loss_mean_arr.npz')

train_loss_arr = train_loss_arr.f.arr_0
valid_loss_arr = valid_loss_arr.f.arr_0
valid_loss_mean_arr = valid_loss_mean_arr.f.arr_0

print("Successfully loaded NPZ")"""

'train_loss_arr = np.load(path + \'train_loss_arr.npz\')\nvalid_loss_arr = np.load(path + \'valid_loss_arr.npz\')\nvalid_loss_mean_arr = np.load(path + \'valid_loss_mean_arr.npz\')\n\ntrain_loss_arr = train_loss_arr.f.arr_0\nvalid_loss_arr = valid_loss_arr.f.arr_0\nvalid_loss_mean_arr = valid_loss_mean_arr.f.arr_0\n\nprint("Successfully loaded NPZ")'

In [86]:
"""valid_loss_arr.shape"""

'valid_loss_arr.shape'

In [87]:
import matplotlib.pyplot as plt
def plot_graph(data, train_or_valid):
    # summarize history for loss
    plt.clf()
    plt.plot(data, color='blue')
    plt.title('model ' + train_or_valid + ' loss')
    plt.ylabel('loss')
    plt.xlabel('epoch')
    plt.legend([train_or_valid], loc='upper left')
    plt.yscale('log')
    # plt.savefig(newpath + '/' + 'train_loss.png')

In [90]:
"""plot_graph(train_loss_arr, 'train')"""

"plot_graph(train_loss_arr, 'train')"

In [91]:
"""plot_graph(valid_loss_arr[0].reshape(-1,), 'valid')"""

"plot_graph(valid_loss_arr[0].reshape(-1,), 'valid')"

In [92]:
"""plot_graph(valid_loss_arr[1].reshape(-1,), 'valid')"""

"plot_graph(valid_loss_arr[1].reshape(-1,), 'valid')"

In [93]:
"""plot_graph(valid_loss_arr[2].reshape(-1,), 'valid')"""

"plot_graph(valid_loss_arr[2].reshape(-1,), 'valid')"

In [94]:
"""plot_graph(valid_loss_arr[3].reshape(-1,), 'valid')"""

"plot_graph(valid_loss_arr[3].reshape(-1,), 'valid')"

In [95]:
"""plot_graph(valid_loss_arr[4].reshape(-1,), 'valid')"""

"plot_graph(valid_loss_arr[4].reshape(-1,), 'valid')"

In [96]:
"""plot_graph(valid_loss_arr[5].reshape(-1,), 'valid')"""

"plot_graph(valid_loss_arr[5].reshape(-1,), 'valid')"

In [97]:
"""plot_graph(valid_loss_mean_arr, 'valid')"""

"plot_graph(valid_loss_mean_arr, 'valid')"

## Testing outputs of model

In [98]:
ACTIONS_LIST = get_actions_list(zigzag=zigzag)

In [99]:
num_of_frames, frames, actions, rewards, lifes = load_npz(data_path, match)

Successfully loaded NPZ.


In [109]:
num_of_frames_arr = []
frames_arr = []
actions_arr = []

start_frame = 1
end_frame = 1000

for m in range(start_match, end_match + 1):
    
    num_of_frames, frames, actions, rewards, lifes = load_npz(data_path, m)
    frames = frames[start_frame - 1:end_frame]
    actions = actions[start_frame - 1:end_frame]
    
    action_one_hot = [prepare_action_data(i, ACTIONS_LIST) for i in actions]
    actions = np.array(action_one_hot)
    actions = actions.reshape(len(actions), -1)
    
    frames_arr.append(frames)
    actions_arr.append(actions)
    num_of_frames_arr.append(end_frame - start_frame + 1) 

X_train = np.array(frames_arr)/255
Y_train = np.array(actions_arr)
num_of_frames_arr = np.array(num_of_frames_arr)

X_train = torch.tensor(X_train).float()
Y_train = torch.tensor(Y_train).float()

Successfully loaded NPZ.


ValueError: 0 is not in list

In [ ]:
X_train = frames.reshape(1, len(frames) ,-1)/255

In [64]:
ACTIONS_LIST

[2, 3]

In [65]:
actions

array([2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
       3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
       3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
       3, 3, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2], dtype=int64)

In [66]:
action_one_hot = [prepare_action_data(i, ACTIONS_LIST) for i in actions]
Y_train = np.array(action_one_hot)
Y_train = Y_train.reshape(1, len(Y_train), -1)

In [67]:
X_train = torch.tensor(X_train).float()
Y_train = torch.tensor(Y_train).float()

In [68]:
hx = torch.zeros(1, hidden_neurons)
cx = torch.zeros(1, hidden_neurons)
out_arr = []
for i in range(1000):
    step_input = X_train[0][i]
    step_input = step_input.reshape(1, -1)
    hx, cx = lstmcell(step_input, (hx, cx))
    out = linear(hx)
    out = output(out)
    out_arr.append(out)

In [69]:
Y_train = Y_train.reshape(-1, len(ACTIONS_LIST))

In [70]:
acertou = 0
errou = 0
for i in range(1000):
    if torch.argmax(Y_train[i]) == torch.argmax(out_arr[i]):
        acertou += 1
    else:
        errou += 1 

In [71]:
Y_train

tensor([[1., 0.],
        [1., 0.],
        [1., 0.],
        [1., 0.],
        [1., 0.],
        [1., 0.],
        [1., 0.],
        [1., 0.],
        [1., 0.],
        [1., 0.],
        [1., 0.],
        [1., 0.],
        [1., 0.],
        [1., 0.],
        [1., 0.],
        [1., 0.],
        [1., 0.],
        [1., 0.],
        [1., 0.],
        [1., 0.],
        [1., 0.],
        [1., 0.],
        [1., 0.],
        [1., 0.],
        [1., 0.],
        [1., 0.],
        [1., 0.],
        [1., 0.],
        [1., 0.],
        [1., 0.],
        [0., 1.],
        [0., 1.],
        [0., 1.],
        [0., 1.],
        [0., 1.],
        [0., 1.],
        [0., 1.],
        [0., 1.],
        [0., 1.],
        [0., 1.],
        [0., 1.],
        [0., 1.],
        [0., 1.],
        [0., 1.],
        [0., 1.],
        [0., 1.],
        [0., 1.],
        [0., 1.],
        [0., 1.],
        [0., 1.],
        [0., 1.],
        [0., 1.],
        [0., 1.],
        [0., 1.],
        [0., 1.],
        [0

In [72]:
acertou

120

In [73]:
errou

0

In [74]:
acertou/(acertou + errou)

1.0

In [ ]:
loss_

## Play Gym Enduro

In [17]:
import gym
import time
from PIL import Image

In [18]:
if zigzag:
        
    ACTIONS = {
        "right": 2,
        "left": 3,
    }

else:

    ACTIONS = {
        "noop": 0,
        "accelerate": 1,
        "right": 2,
        "left": 3,
        "break": 4,
        "right_break": 5,
        "left_break": 6,
        "right_accelerate": 7,
        "left_accelerate": 8,
    }

In [19]:
y_min, y_max, x_min, x_max = 25, 195, 20, 140
shape_of_single_frame = (1, (y_max-y_min),(x_max-x_min))

In [20]:
sleep_time = 0.05

In [23]:
env = gym.make("Enduro-v0")
frame = env.reset()
reward, action, done, info = 0, 0, False, {'ale.lives': 0}

hx = torch.zeros(1, hidden_neurons)
cx = torch.zeros(1, hidden_neurons)

env.render()
time.sleep(10)

for _ in range(1000):
    
    time.sleep(sleep_time)
    env.render()
    
    frame = frame[y_min:y_max, x_min:x_max]

    frame = Image.fromarray(frame)
    frame = frame.convert("L")
    
    frame = np.asarray(frame)
    frame = frame.reshape(1, -1)
    frame = torch.tensor(frame)/255
    
    hx, cx = lstmcell(frame, (hx, cx))
    out = linear(hx)
    action = output(out)
    
    action = list(ACTIONS.values())[torch.argmax(action, axis=1)]
    print(action)
    frame, reward, done, info = env.step(action)

<ipython-input-23-2ac00970ed70>:27: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  action = output(out)


0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
0
0
0
0
0
0
0
0
0
0
0
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
